# Exercises using TESPy

In [ ]:
from tespy.components import Compressor, SimpleHeatExchanger, Valve, CycleCloser
from tespy.connections import Connection
from tespy.tools.helpers import UserDefinedEquation
from tespy.networks import Network


nw = Network(T_unit="C", p_unit="bar", h_unit="kJ / kg", iterinfo=False)

cc = CycleCloser("cycle closer")
ev = SimpleHeatExchanger("evaporator")
cp = Compressor("compressor")
cd = SimpleHeatExchanger("condenser")
va = Valve("valve")

c1 = Connection(cc, "out1", ev, "in1", label="1")
c2 = Connection(ev, "out1", cp, "in1", label="2")
c3 = Connection(cp, "out1", cd, "in1", label="3")
c4 = Connection(cd, "out1", va, "in1", label="4")
c0 = Connection(va, "out1", cc, "in1", label="0")

nw.add_conns(c0, c1, c2, c3, c4)

In [ ]:
t_2 = 10
t_4 = 60
heat = -1e6
fluid = "R290"
eta_s = 0.8

c2.set_attr(fluid={fluid: 1}, x=1, T=t_2)
c4.set_attr(x=0, T=t_4)
cp.set_attr(eta_s=eta_s)
ev.set_attr(pr=1)
cd.set_attr(pr=1, Q=heat)

In [ ]:
nw.solve("design")

In [ ]:
cop = abs(cd.Q.val) / cp.P.val
cop
cp.P.val

In [ ]:
cp.set_attr(P=0.2e6)
cd.set_attr(Q=None)
nw.solve("design")
cd.Q.val

In [ ]:
cd.set_attr(Q=-1e6)
c4.set_attr(T=None)
nw.solve("design")
c4.T.val

In [ ]:
def cop_ude(ude):
    c1, c2, c3 = ude.conns
    cop = ude.params["cop"]
    return (c2.h.val_SI - c3.h.val_SI) - cop * (c2.h.val_SI - c1.h.val_SI)


def cop_jacobian(ude):
    c1, c2, c3 = ude.conns
    if c1.h.is_var:
        ude.jacobian[c1.h.J_col] = cop
    if c2.h.is_var:
        ude.jacobian[c2.h.J_col] = 1 - cop
    if c3.h.is_var:
        ude.jacobian[c3.h.J_col] = -1

In [ ]:
cd.set_attr(Q=None)


ude = UserDefinedEquation("cop-ude", cop_ude, cop_jacobian, [c2, c3, c4], {"cop": 5})
nw.add_ude(ude)

nw.solve("design")

In [ ]:
c4.T.val - c2.T.val

In [ ]:
c2.set_attr(T=None)
c1.set_attr(x=0.35)

In [ ]:
nw.solve("design")
c4.T.val - c2.T.val

In [ ]:
cp.set_attr(eta_s=None, P=None)
cd.set_attr(Q=-1e6)
c1.set_attr(m=3.5)
nw.solve("design")
c4.T.val, c2.T.val
cp.eta_s.val